## Giới thiệu

Trong bài này, tôi sẽ giới thiệu thuật toán đầu tiên trong Classification có tên là Perceptron Learning Algorithm (PLA) hoặc đôi khi được viết gọn là Perceptron. Về tên gọi của thuật toán này, tôi sẽ nêu ở phần cuối của bài. Tạm thời, bạn đọc chỉ cần biết rằng perceptron là một khái niệm dùng trong mạng dây thần kinh của con người. 

Perceptron là một thuật toán Classification cho trường hợp đơn giản nhất: chỉ có hai class (lớp) và cũng chỉ hoạt động được trong một trường hợp rất cụ thể. Tuy nhiên, nó là nền tảng cho một mảng lớn quan trọng của Machine Learning là Neural Networks và sau này là Deep Learning. Easy first!

Giải sử chúng ta có hai tập hợp dữ liệu đã được gán nhãn được minh hoạ như Hình 1 bên trái dưới đây. Hai tập hợp của chúng ta là tập các điểm màu xanh và tập các điểm màu đỏ. Bài toán đặt ra là: từ dữ liệu của hai tập được gán nhãn chúng ta có, hãy xây dựng một _classifier_ (bộ phân lớp) để khi có một điểm dữ liệu hình tam giác màu xám mới, ta có thể dự đoán được màu (nhãn) của nó. Mỗi tập/nhóm dữ liệu đã được gán nhãn được gọi là một _class_.

<table width = "100%" style = "border: 0px solid white">
   <tr >
        <td width="40%" style = "border: 0px solid white"> 
        <img style="display:block;" width = "100%" src = "/assets/pla/pla1.png">
         </td>
        <td width="40%" style = "border: 0px solid white">
        <img style="display:block;" width = "100%" src = "/assets/pla/pla2.png">
        </td>
    </tr>
</table> 

Hiểu theo một cách khác, chúng ta cần tìm _lãnh thổ_ của mỗi class sao cho, với mỗi một điểm mới, ta chỉ cần xác định xem nó nằm vào lãnh thổ của class nào rồi quyết định nó thuộc class đó. Để tìm _lãnh thổ_ của mỗi class, chúng ta cần đi tìm biên giới (boundary) giữa hai _lãnh thổ_ này. Vậy bài toán classification có thể coi là bài toán đi tìm boundary giữa các class. Và boundary đơn giản nhât trong không gian hai chiều là một đường thằng, trong không gian ba chiều là một mặt phẳng, trong không gian nhiều chiều là một siêu mặt phẳng (hyperplane) (tôi gọi chung những boundary này là _đường phẳng_). Những boundary phẳng này được coi là đơn giản vì nó có thể biểu diễn dưới dạng toán học bằng một hàm số đơn giản có dạng tuyến tính, tức linear. Tất nhiên, chúng ta đang giả sử một cách đơn giản rằng tồn tại một đường phẳng để có thể phân định _lãnh thổ_ của hai class. Hình 1 bên phải minh họa một đường thẳng phân chia hai class trong mặt phẳng. Phần có nền màu xanh được coi là _lãnh thổ_ của lớp xanh, phần có nên màu đỏ được coi là _lãnh thổ_ của lớp đỏ. Trong trường hợp này, điểm dữ liệu mới hình tam giác được phân vào class đỏ. 

### Bài toán Perceptron 
Bài toán Perceptron được phát biểu như sau: _Cho hai class được gán nhãn, hãy tìm một đường phẳng sao cho toàn bộ các điểm thuộc class 1 nằm về 1 phía, toàn bộ các điểm thuộc class 2 nằm về phía còn lại của đường phẳng đó. Với giả định rằng tồn tại một đường phẳng như thế._

Nếu tồn tại một đường phẳng phân chia hai class thì ta gọi hai class đó là _linear separable_. 

## Thuật toán Perceptron (PLA)
Cũng giống như các thuật toán lặp trong [K-means Clustering](/2017/01/01/kmeans/) và [Gradient Descent](/2017/01/12/gradientdescent/), ý tưởng cơ bản của PLA là xuất phát từ một nghiệm dự đoán nào đó, qua mỗi vòng lặp, nghiệm sẽ được cập nhật tới một ví trí tốt hơn. Việc cập nhật này dựa trên việc giảm giá trị của một hàm mất mát nào đó. 

### Một số ký hiệu
Giả sử \\(\mathbf{X} = [\mathbf{x}_1, \mathbf{x}_2, \dots, \mathbf{x}_N] \in \mathbb{R}^{d \times N}\\) là ma trận chứa các điểm dữ liệu mà mỗi cột \\(\mathbf{x}_i \in \mathbb{R}^{d\times 1}\\) là một điểm dữ liệu trong không gian \\(d\\) chiều. (_Chú ý: khác với các bài trước tôi thường dùng các vector hàng để mô tả dữ liệu, trong bài này tôi dùng vector cột để biểu diễn. Việc biểu diễn dữ liệu ở dạng hàng hay cột tùy thuộc vào từng bài toán, miễn sao cách biễu diễn toán học của nó khiến cho người đọc thấy dễ hiểu_).

Giả sử thêm các nhãn tương ứng với từng điểm dữ liệu được lưu trong một vector hàng \\(\mathbf{y} = [y_1, y_2, \dots, y_N] \in \mathbb{R}^{1\times d}\\), với \\(y_i = 1\\) nếu \\(\mathbf{x}_i\\) thuộc class 1 (xanh) và \\(y_i = -1\\) nếu \\(\mathbf{x}_i\\) thuộc class 2 (đỏ).

Tại một thời điểm, giả sử ta tìm được boundary là đường phẳng có phương trình:
\\[
f_{\mathbf{w}}(\mathbf{x}) = w_1x_1 + w_2x_2 + \dots + w_Nx_N + w_0 = 
\\]
\\[
=\mathbf{w}^T\mathbf{\bar{x}} = 0
\\]

với \\(\mathbf{\bar{x}}\\) là điểm dữ liệu mở rộng bằng cách thêm phần tử 1 lên trước vector \\(\mathbf{x}\\) tương tự như trong [Linear Regression](/2016/12/28/linearregression/). Và từ đây, khi nói \\(\mathbf{x}\\), tôi cũng ngầm hiểu là điểm dữ liệu mở rộng.

Để cho đơn giản, chúng ta hãy cùng làm việc với trường hợp mỗi điểm dữ liệu có số chiều \\(d = 2\\). Giả sử đường thẳng \\(w_1 x_1 + w_2 x_2 + w_0 = 0\\) chính là nghiệm cần tìm như Hình 2 dưới đây:

<div class="imgcap">
<img src ="\assets\pla\pla4.png" align = "center" width = "400">
</div> 

Nhận xét rằng các điểm nằm về cùng 1 phía so với đường thẳng này sẽ làm cho hàm số \\(f_{\mathbf{w}}(\mathbf{x})\\) mang cùng dấu. Chỉ cần đổi dấu của \\(\mathbf{w}\\) nếu cần thiết, ta có thể giả sử các điểm nằm trong nửa mặt phẳng nền xanh mang dấu dương (+), các điểm nằm trong nửa mặt phẳng nền đỏ mang dấu ấm (-). Các dấu này cũng tương đương với nhãn \\(y\\) của mỗi class. Vậy nếu \\(\mathbf{w}\\) là một nghiệm của bài toán Perceptron, với một điểm dữ liệu mới \\(\mathbf{x}\\) chưa được gán nhãn, ta có thể xác định class của nó bằng phép toán đơn giản như sau:
\\[
\text{label}(\mathbf{x}) = 1 ~\text{if}~ \mathbf{w}^T\mathbf{x} \geq 0, \text{otherwise} -1
\\]

Ngắn gọn hơn: 
\\[
\text{label}(\mathbf{x}) = \text{sgn}(\mathbf{w}^T\mathbf{x})
\\]
trong đó, \\(\text{sgn}\\) là hàm xác định dấu, với giả sử rằng \\(\text{sgn}(0) = 1\\).

### Xây dựng hàm mất mát
Tiếp theo, chúng ta cần xây dựng hàm mất mát với tham số \\(\mathbf{w}\\) bất kỳ. Vẫn trong không gian hai chiều, giả sử đường thẳng \\(w_1x_1 + w_2x_2 + w_0 = 0\\) được cho như Hình 3 dưới đây:
<div class="imgcap">
<img src ="\assets\pla\pla3.png" align = "center" width = "400">
</div> 
Trong trường hợp này, các điểm được khoanh tròn là các điểm bị misclassified (phân lớp lỗi). Điều chúng ta mong muốn là không có điểm nào bị misclassified. Hàm mất mát đơn giản nhất chúng ta nghĩ đến là hàm _đếm_ số lượng các điểm bị misclassied và tìm cách tối thiểu hàm số này:
\\[
J_1(\mathbf{w}) = \sum_{\mathbf{x}_i \in \mathcal{M}} (-y_i\text{sgn}(\mathbf{w}^T\mathbf{x_i}))
\\]

trong đó \\(\mathcal{M}\\) là tập hợp các điểm bị misclassifed. Với mỗi điểm \\(\mathbf{x}_i \in \mathcal{M}\\), vì điểm này bị misclassified nên \\(y_i\\) và \\(\text{sgn}(\mathbf{w}^T\mathbf{x})\\) khác nhau và vì thế \\(-y_i\text{sgn}(\mathbf{w}^T\mathbf{x_i}) = 1 \\). Vậy \\(J\_1(\mathbf{w})\\) chính là hàm _đếm_ số lượng các điểm bị misclassified. Khi hàm số này đạt giá trị nhỏ nhất bằng 0 thì ta không còn điểm nào bị misclassified. 

Một điểm quan trọng, hàm số này là rời rạc, không tính được đạo hàm theo \\(\mathbf{w}\\) nên rất khó tối ưu. Chúng ta cần tìm một hàm mất mát khác để việc tối ưu khả thi hơn.

Xét hàm mất mát sau đây: 
\\[
J(\mathbf{w}) = \sum_{\mathbf{x}_i \in \mathcal{M}} (-y_i\mathbf{w}^T\mathbf{x_i})
\\]
Hàm \\(J()\\) khác một chút với hàm \\(J\_1()\\) ở việc bỏ đi hàm \\(\text{sgn}\\). Nhận xét rằng khi một điểm misclassified \\(\mathbf{x}_i\\) nằm càng xa boundary thì giá trị \\(-y_i\mathbf{w}^T\mathbf{x_i}\\) sẽ càng lớn, nghĩa là sự sai lệch càng lớn. Giá trị nhỏ nhất của hàm mất mát này cũng bằng 0 nếu không có điểm nào bị misclassifed. 

Tại một thời điểm, nếu chúng ta chỉ quan tâm tới các điểm bị misclassified thì hàm số \\(J(\mathbf{w})\\) khả vi (tính được đạo hàm), vậy chúng ta có thể dụng [Gradient Descent](/2017/01/12/gradientdescent/) hoặc [Stochastic Gradient Descent (SGD)](/2017/01/16/gradientdescent2/#-stochastic-gradient-descent) để tối ưu hàm mất mát này. Với lợi ích của SGD cho các bài toán large-scale, chúng ta sẽ làm theo thuật toán này. 